In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
pd.set_option('display.max_columns',None)



from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(15,5)
%config InlineBackend.figure_format = 'png'
%matplotlib inline

Загружаем данные в датафреймы 

In [2]:
df_checks_with_cat = pd.read_pickle("./df_checks_with_cat.pkl")


In [3]:
df_shops = pd.read_pickle("./df_shops.pkl")


In [4]:
df_checks_with_cat.head()

,CheckUID,id_tt_cl,id_tov_cl,Quantity,week,BaseSum,hour,name_gr,name_gr2
0,32C04617-1DA6-EC11-9064-005056A7A8DF,16061,25528,1.0,11,422.73,20,Овощи. Фрукты. Грибы. Зелень,Авокадо
1,7CC04B95-79AA-EC11-9064-005056A7A8DF,11846,1609,2.0,12,67.62,10,Молочная Продукция,Лакомства молочные
2,7BC04CF0-8AA6-EC11-9064-005056A7A8DF,11160,49220,1.0,11,124.17,10,Пространство ВкусВилл,Продажа услуг
3,75C04F15-4C9A-EC11-9064-005056A7A8DF,14258,55220,1.0,9,69.09,19,Молочная Продукция,Йогурты питьевые
4,54C0631E-D2AD-EC11-9064-005056A7A8DF,12853,32931,1.0,12,49.09,16,Хлеб,Хлеб на закваске


In [5]:
df_checks_with_cat['id_tt_cl'].nunique()

159

Строим пивот таблицу по группам

In [6]:
df_category_pivot_sum = df_checks_with_cat.pivot_table(columns=['name_gr'],index=['id_tt_cl'],values = ['BaseSum'],aggfunc= 'sum',fill_value=0).reset_index()


Исправляем кривое форматирование пивот таблицы

In [7]:

df_category_pivot_sum = pd.concat([df_category_pivot_sum['id_tt_cl'].to_frame(),df_category_pivot_sum.loc[:,'BaseSum']], axis=1)

Добавляем по каждому магазину сумму проданных итемов -- df_category_pivot_sum

In [8]:
df_category_pivot_sum['overal_sum'] = df_category_pivot_sum.drop('id_tt_cl',axis=1).sum(axis=1)

Добавляем по каждому магазину количество проданных итемов в процентах по каждой категории -- df_category_pivot_sum3

In [9]:
df_category_pivot_sum3 = df_category_pivot_sum.copy()

In [10]:
for column in df_category_pivot_sum3.drop(['id_tt_cl','overal_sum'],axis=1):
    df_category_pivot_sum3[column] = df_category_pivot_sum3[column].div(df_category_pivot_sum3['overal_sum']/100)

In [11]:
del df_category_pivot_sum3['overal_sum']

In [12]:
df_category_pivot_sum3 = df_category_pivot_sum3.set_index('id_tt_cl')

In [27]:
df_worst_milk_sales =  df_category_pivot_sum3.nsmallest(n = 30 ,columns= 'Молочная Продукция')
df_worst_milk_sales = df_worst_milk_sales ['Молочная Продукция'].to_frame()

In [28]:
#Create Dict id_tt_cl => Quality
dic_shops_qual = df_shops['quality'].to_dict()
dic_shops_size = df_shops['size'].to_dict()

In [29]:
df_worst_milk_sales['quality'] = df_worst_milk_sales.index
df_worst_milk_sales['size'] = df_worst_milk_sales.index

In [30]:
df_worst_milk_sales.replace({"quality": dic_shops_qual},inplace=True)
df_worst_milk_sales.replace({"size": dic_shops_size},inplace=True)

In [44]:
print('Bad shops : ',df_worst_milk_sales[df_worst_milk_sales['quality'] < 0].shape[0])
print('Average shops : ',df_worst_milk_sales[(df_worst_milk_sales['quality'] > 9) & (df_worst_milk_sales['quality'] < 100)].shape[0])
print('Good shops : ',df_worst_milk_sales[(df_worst_milk_sales['quality'] > 0) & (df_worst_milk_sales['quality'] < 4)].shape[0])

Bad shops :  19
Average shops :  6
Good shops :  5


In [45]:

df_worst_milk_sales.to_pickle("./df_worst_milk_sales.pkl")

In [61]:
df_worst_milk_sales[df_worst_milk_sales['quality'] < 0].to_pickle("./df_worst_milk_sales_bad_shops.pkl")

In [56]:
df_best_milk_sales =  df_category_pivot_sum3.nlargest(n = 30 ,columns= 'Молочная Продукция')
df_best_milk_sales = df_best_milk_sales ['Молочная Продукция'].to_frame()

In [57]:
df_best_milk_sales['quality'] = df_best_milk_sales.index
df_best_milk_sales['size'] = df_best_milk_sales.index

In [58]:
df_best_milk_sales.replace({"quality": dic_shops_qual},inplace=True)
df_best_milk_sales.replace({"size": dic_shops_size},inplace=True)

In [59]:
df_best_milk_sales.to_pickle("./df_best_milk_sales.pkl")

In [60]:

print('Bad shops : ',df_best_milk_sales[df_best_milk_sales['quality'] < 0].shape[0])
print('Average shops : ',df_best_milk_sales[(df_best_milk_sales['quality'] > 9) & (df_best_milk_sales['quality'] < 100)].shape[0])
print('Good shops : ',df_best_milk_sales[(df_best_milk_sales['quality'] > 0) & (df_best_milk_sales['quality'] < 4)].shape[0])

Bad shops :  6
Average shops :  9
Good shops :  15


In [63]:
df_best_milk_sales[(df_best_milk_sales['quality'] > 0) & (df_best_milk_sales['quality'] < 4)].to_pickle("./df_best_milk_sales_good_shops.pkl")